© 2025 Vanargo · Лицензия: MIT. См. файл `LICENSE` в корне репозитория.

# --- 01. Data loading and EDA --- #

Загрузка, чистка, исследовательский анализ и создание признаков для датасета Adult (Census Income)

In [ ]:
# --- Импорты и базовый стиль --- #

import logging
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import chi2_contingency

# графики: единый аккуратный стиль #
plt.rcParams.update(
    {
        "figure.figsize": (8, 4),
        "axes.spines.top": False,
        "axes.spines.right": False,
    }
)
sns.set_context("notebook")

# тише логгеры популярных библиотек #
for name in ("lightgbm", "xgboost", "matplotlib", "numba"):
    logging.getLogger(name).setLevel(logging.ERROR)

# фильтры предупреждений, релевантные ноутбуку #
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=".*Glyph .* missing from current font.*")

# контролируемая случайность для воспроизводимости #
RNG = np.random.default_rng(42)

In [ ]:
# --- Project paths bootstrap --- #

import sys

# определение корня проекта относительно текущего ноутбука #
ROOT = Path.cwd().resolve().parents[0]
DATA_DIR = ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR = DATA_DIR / "reports"

# добавление корня в sys.path, если его там нет #
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print(f"[ok] project root: {ROOT}")
print(f"[ok] data dir: {DATA_DIR}")

In [ ]:
# --- Notebook preamble: UX, magics, helpers --- #

# IPython magics #
%load_ext autoreload
%autoreload 2
%matplotlib inline

# display options #
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")

# reproducibility #
try:
    _ = RNG
except NameError:
    RNG = np.random.default_rng(42)

# figures directory for this notebook #
FIG_DIR_01 = REPORTS_DIR / "figures_01"
FIG_DIR_01.mkdir(parents=True, exist_ok=True)


def save_fig(fig, name: str, dpi: int = 150) -> Path:
    """
    Save matplotlib figure to reports/figures_01 as <name>.png.
    """
    p = FIG_DIR_01 / f"{name}.png"
    fig.savefig(p, dpi=dpi, bbox_inches="tight")
    print(f"[saved] {p}")
    return p


def log_df(df: pd.DataFrame, name: str) -> None:
    """
    Compact dataframe log: shape and column list.
    """
    print(f"[{name}] shape={df.shape} cols={list(df.columns)}")


# sanity print #
print(f"[env] pandas {pd.__version__} | numpy {np.__version__} | seaborn {sns.__version__}")
print(f"[figures] {FIG_DIR_01}")

In [ ]:
# --- Загрузка данных Adult (train/test) --- #

# пути #
TRAIN_PATH = RAW_DIR / "adult.data"
TEST_PATH = RAW_DIR / "adult.test"

# имена колонок из документации UCI #
COLS = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "income",
]

# общие параметры чтения #
READ_OPTS = dict(
    names=COLS,
    na_values=["?", " ?"],
    skipinitialspace=True,
    # читаем как строки, затем приводим типы явно
    dtype=str,
)

# train #
df_train = pd.read_csv(TRAIN_PATH, **READ_OPTS)
df_train["source"] = "train"

# test (в файле первая строка — заголовок, пропускаем) #
df_test = pd.read_csv(TEST_PATH, **READ_OPTS, skiprows=1)
df_test["source"] = "test"

# объединение #
df = pd.concat([df_train, df_test], ignore_index=True)

# чистка пробелов и целевых значений #
df = df.apply(lambda c: c.str.strip() if c.dtype == "object" else c)
df["income"] = df["income"].replace({"<=50K.": "<=50K", ">50K.": ">50K"})

# явное приведение типов для числовых признаков #
NUMERIC_COLS = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
for c in NUMERIC_COLS:
    # NaN если мусор
    df[c] = pd.to_numeric(df[c], errors="coerce")

# лог и предпросмотр #
log_df(df, "adult_full")
df.head()

In [ ]:
# --- Первичный обзор --- #

print("Shape:", df.shape)

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
obj_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

if num_cols:
    display(df[num_cols].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]).T)
else:
    print("[info] numeric columns not found")

if obj_cols:
    display(df[obj_cols].describe().T)
else:
    print("[info] object/category columns not found")

In [ ]:
# --- Распределения числовых признаков --- #

num_cols = df.select_dtypes(include=np.number).columns.tolist()
print(f"[ok] numeric columns: {len(num_cols)}")

if not num_cols:
    print("[warn] numeric columns not found")
else:
    # индивидуальные гистограммы #
    for col in num_cols:
        fig, ax = plt.subplots(figsize=(6, 3))
        sns.histplot(data=df, x=col, kde=True, bins=30, ax=ax, color="steelblue")
        ax.set_title(f"Распределение {col}")
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
        ax.grid(alpha=0.3)
        plt.tight_layout()
        save_fig(fig, f"hist_{col}")
        plt.close(fig)

    # сводный график в одной сетке #
    ncols = 3
    nrows = int(np.ceil(len(num_cols) / ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 4, nrows * 3))
    axes = axes.flatten()

    for i, col in enumerate(num_cols):
        sns.histplot(data=df, x=col, kde=True, bins=30, ax=axes[i], color="steelblue")
        axes[i].set_title(col)
        axes[i].grid(alpha=0.3)

    # отключение пустых осей #
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    save_fig(fig, "hist_numeric_all")
    plt.close(fig)

In [ ]:
# --- Корреляция между числовыми признаками --- #

num_cols = df.select_dtypes(include=np.number).columns.tolist()

if not num_cols:
    print("[warn] numeric columns not found, correlation skipped")
else:
    corr = df[num_cols].corr(method="pearson")

    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(
        corr,
        cmap="vlag",
        center=0,
        annot=True,
        fmt=".2f",
        linewidths=0.5,
        cbar_kws={"shrink": 0.8},
        ax=ax,
    )
    ax.set_title("Корреляция числовых признаков (Pearson)")
    plt.tight_layout()
    save_fig(fig, "corr_numeric")
    plt.close(fig)

    display(corr)

In [ ]:
# --- Ассоциации между категориальными признаками (Cramér’s V) --- #


cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()


def cramers_v(x: pd.Series, y: pd.Series) -> float:
    """
    Вычисляет Cramér’s V для двух категориальных переменных.
    """
    tbl = pd.crosstab(x, y, dropna=False)
    chi2, _, _, _ = chi2_contingency(tbl, correction=False)
    n = tbl.values.sum()
    phi2 = chi2 / n
    r, k = tbl.shape
    phi2corr = max(0, phi2 - (k - 1) * (r - 1) / (n - 1))
    rcorr = r - (r - 1) ** 2 / (n - 1)
    kcorr = k - (k - 1) ** 2 / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))


if len(cat_cols) < 2:
    print("[info] not enough categorical features for Cramér’s V matrix")
else:
    # матрица Cramér’s V #
    cramers_mat = pd.DataFrame(
        np.zeros((len(cat_cols), len(cat_cols))), index=cat_cols, columns=cat_cols
    )

    for i, col1 in enumerate(cat_cols):
        for j, col2 in enumerate(cat_cols):
            if i >= j:
                continue
            v = cramers_v(df[col1], df[col2])
            cramers_mat.loc[col1, col2] = cramers_mat.loc[col2, col1] = v

    # визуализация #
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(
        cramers_mat,
        cmap="crest",
        vmin=0,
        vmax=1,
        annot=True,
        fmt=".2f",
        square=True,
        cbar_kws={"shrink": 0.7},
        ax=ax,
    )
    ax.set_title("Cramér’s V между категориальными признаками")
    plt.tight_layout()
    save_fig(fig, "cramers_v_matrix")
    plt.close(fig)

    display(cramers_mat.round(2))

In [ ]:
# --- Распределения категориальных признаков --- #

cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
print(f"[ok] categorical columns: {len(cat_cols)}")

if not cat_cols:
    print("[warn] no categorical columns found")
else:
    for col in cat_cols:
        # если кардинальность высокая, строим топ-20 #
        n_unique = df[col].nunique(dropna=True)
        if n_unique > 20:
            top_vals = df[col].value_counts(dropna=False).head(20).sort_values(ascending=False)
            data_plot = pd.DataFrame({"value": top_vals.index, "count": top_vals.values})
            title = f"{col} (top 20 из {n_unique})"
        else:
            data_plot = df[[col]].copy()
            title = col

        fig, ax = plt.subplots(figsize=(7, 3))
        sns.countplot(
            data=data_plot, x=col if n_unique <= 20 else "value", ax=ax, color="steelblue"
        )
        ax.set_title(f"Распределение: {title}")
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
        ax.tick_params(axis="x", rotation=45)
        ax.grid(alpha=0.3)
        plt.tight_layout()
        save_fig(fig, f"bar_{col}")
        plt.close(fig)

In [ ]:
# --- Категориальные признаки vs целевая переменная (income) --- #

target_col = "income"
cat_cols = [c for c in df.select_dtypes(include=["object", "category"]).columns if c != target_col]

if target_col not in df.columns:
    print(f'[warn] target "{target_col}" not found')
elif not cat_cols:
    print("[warn] no categorical columns found")
else:
    for col in cat_cols:
        n_unique = df[col].nunique(dropna=True)
        if n_unique > 20:
            print(f"[skip] {col}: {n_unique} unique values")
            continue

        fig, ax = plt.subplots(figsize=(7, 3))
        sns.countplot(data=df, x=col, hue=target_col, ax=ax, palette="Set2")
        ax.set_title(f"{col} vs {target_col}")
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
        ax.tick_params(axis="x", rotation=45)
        ax.legend(title=target_col, loc="upper right")
        ax.grid(alpha=0.3)
        plt.tight_layout()
        save_fig(fig, f"bar_{col}_vs_{target_col}")
        plt.close(fig)

In [ ]:
# --- Числовые признаки vs целевая переменная (income) --- #

target_col = "income"
num_cols = df.select_dtypes(include=np.number).columns.tolist()

if target_col not in df.columns:
    print(f'[warn] target "{target_col}" not found')
elif not num_cols:
    print("[warn] numeric columns not found")
else:
    for col in num_cols:
        fig, ax = plt.subplots(figsize=(6, 3))
        sns.boxplot(
            data=df,
            x=target_col,
            y=col,
            hue=target_col,  # фиксация FutureWarning
            legend=False,  # отключение легенды (она дублирует подписи)
            palette="Set2",
            showfliers=False,
            ax=ax,
        )
        ax.set_title(f"{col} vs {target_col}")
        ax.set_xlabel(target_col)
        ax.set_ylabel(col)
        ax.grid(alpha=0.3)
        plt.tight_layout()
        save_fig(fig, f"box_{col}_vs_{target_col}")
        plt.close(fig)

In [ ]:
# --- Анализ пропусков --- #

# подсчет пропусков #
na_stats = df.isna().sum()
na_stats = na_stats[na_stats > 0].sort_values(ascending=False)

if na_stats.empty:
    print("[ok] пропусков не обнаружено")
else:
    na_df = pd.DataFrame(
        {"missing_count": na_stats, "missing_pct": (na_stats / len(df) * 100).round(2)}
    )
    display(na_df)

    # визуализация #
    fig, ax = plt.subplots(figsize=(7, 3))
    sns.barplot(data=na_df.reset_index(), x="index", y="missing_pct", color="steelblue", ax=ax)
    ax.set_title("Доля пропусков по признакам (%)")
    ax.set_xlabel("Feature")
    ax.set_ylabel("Missing (%)")
    ax.tick_params(axis="x", rotation=45)
    ax.grid(alpha=0.3)
    plt.tight_layout()
    save_fig(fig, "missing_values")
    plt.close(fig)

In [ ]:
# --- Выбросы по IQR и бинарные флаги --- #

NUM_COLS = df.select_dtypes(include=np.number).columns.tolist()
OUTLIER_PREFIX = "is_outlier_"


def iqr_flags(s: pd.Series, k: float = 1.5) -> pd.Series:
    """Возвращение 0/1 флага выброса по правилу IQR."""
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    if not np.isfinite(iqr) or iqr == 0:
        return pd.Series(np.zeros(len(s), dtype=int), index=s.index)
    lo, hi = q1 - k * iqr, q3 + k * iqr
    return ((s < lo) | (s > hi)).astype(int)


if not NUM_COLS:
    print("[warn] numeric columns not found, outlier flags skipped")
else:
    for c in NUM_COLS:
        flag_col = f"{OUTLIER_PREFIX}{c}"
        df[flag_col] = iqr_flags(df[c])

    # краткая сводка по доле выбросов #
    flag_cols = [c for c in df.columns if c.startswith(OUTLIER_PREFIX)]
    outlier_share = (df[flag_cols].sum().sort_values(ascending=False) / len(df)).round(4)
    display(outlier_share.to_frame("share"))
    print(f"[ok] flags added: {len(flag_cols)}")

In [ ]:
# --- Пользовательские признаки (feature engineering) --- #


def add_custom_features(dfin: pd.DataFrame) -> pd.DataFrame:
    df_ = dfin.copy()

    # чистый капитал #
    if {"capital_gain", "capital_loss"}.issubset(df_.columns):
        df_["capital_net"] = (df_["capital_gain"].fillna(0) - df_["capital_loss"].fillna(0)).astype(
            float
        )
    else:
        df_["capital_net"] = np.nan

    # полезные отношения #
    if {"hours_per_week", "education_num"}.issubset(df_.columns):
        with np.errstate(divide="ignore", invalid="ignore"):
            df_["hours_per_edu"] = df_["hours_per_week"].astype(float) / df_[
                "education_num"
            ].replace(0, np.nan).astype(float)

    # индикатор ненулевого капитала #
    if "capital_net" in df_.columns:
        df_["has_capital"] = (df_["capital_net"].fillna(0) != 0).astype(int)

    return df_


df = add_custom_features(df)
log_df(df, "adult_with_custom_features")

# быстрый просмотр #
df[
    [
        "capital_gain",
        "capital_loss",
        "capital_net",
        "hours_per_week",
        "education_num",
        "hours_per_edu",
        "has_capital",
    ]
].head()

In [ ]:
# --- Категоризация возраста (age_group) --- #

if "age" not in df.columns:
    print('[warn] "age" not in columns, age_group skipped')
else:
    bins = [0, 24, 34, 44, 54, 64, np.inf]
    labels = ["18–24", "25–34", "35–44", "45–54", "55–64", "65+"]
    df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, right=True, include_lowest=True)
    df["age_group"] = df["age_group"].astype("category")
    print("[ok] age_group created")

In [ ]:
# --- Экспорт EDA-датасета --- #

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
p_parquet = PROCESSED_DIR / "adult_eda.parquet"
p_csv = PROCESSED_DIR / "adult_eda.csv"

# экспорт #
df.to_parquet(p_parquet, index=False)
df.to_csv(p_csv, index=False)

print(f"[saved] {p_parquet}")
print(f"[saved] {p_csv}")

In [ ]:
# --- Экспорт ключевых фигур (контроль) --- #

expected = [
    "hist_numeric_all.png",
    "corr_numeric.png",
    "cramers_v_matrix.png",
    "missing_values.png",
]
# плюс поштучные: hist_*, bar_*, box_* - уже сохранены в предыдущих ячейках #

missing = [name for name in expected if not (FIG_DIR_01 / name).exists()]
if missing:
    print("[warn] missing figures:", missing)
else:
    print("[ok] key figures present")

# --- Итоговые выводы по EDA ---

**Качество данных:**
1. Пропуски сконцентрированы в трех признаках: `workclass`, `occupation`, `native_country` (см. `missing_values.png`). Остальные поля заполнены.
2. Категориальные значения унифицированы, редкие категории встречаются преимущественно в `native_country`. Это важно для группировки "прочие" перед моделированием.
3. Явных дубликатов по ключам не выявлено в рамках текущего обзора.
4. Дисбаланс целевой переменной присутствует: `>50k` заметно ниже `<=50k` (см. `bar_income.png`). Для оценки метрик потребуется стратификация и контроль за baseline.

**Базовые распределения числовых признаков:**
1. Возраст: правосторонний с концентрацией в диапазоне 25-45 лет; высокодоходные чаще встречаются среди 35-55 (см. `hist_age.png`, а также разрез по доходу на `box_age_vs_income.png`).
2. `hours-per-week`: модальный район около 40 часов, "тяжелые хвосты" у сверхурочной занятости; медиана выше у группы `>50k` (см. `hist_hours_per_week.png`, `box_hours_per_week_vs_income.png`).
3. `education_num`: монотонная связь с доходом, более высокие значения характерны для `>50k` (см. `hist_education_num.png`, `box_education_num_vs_income.png`).
4. `capital_gain` и `capital_loss`: крайне разреженные, массив нулей с редкими крупными значениями; наличие ненулевых значений сильно ассоциировано с `>50k` (см. `hist_capital_gain.png`, `hist_capital_loss.png`, `box_capital_gain_vs_income.png`, `box_capital_loss_vs_income.png`).
5. `fnlwgt`: широкое распределение, явной интерпретируемой связи с доходом не наблюдается (см. `hist_fnlwgt.png`, `box_fnlwgt_vs_income.png`).
6. Скалярные корреляции невысокие, но заметны для `education_num`, `hours-per-week`, наличия `capital_gain` (см. `corr_numeric.png`).

**Категориальные признаки: профиль и связь с доходом:**
1. Образование: бакалавриат и выше чаще у `>50k`; неполное среднее и среднее - преимущественно у `<=50k` (см. `bar_education.png`, `bar_education_vs_income.png`).
2. Семейное положение: `Married-civ-spouse` доминирует среди `>50k`; `Never-married` и `Divorced` - среди `<=50k` (см. `bar_marital_status.png`, `bar_marital_status_vs_income.png`).
3. Роль в домохозяйстве (`relationship`): категории, коррелирующие с наличием супруг(-и)/главы домохозяйства, чаще принадлежат `>50k` (см. `bar_relationship.png`, `bar_relationship_vs_income.png`).
4. Профессии: `Exec-managerial`, `Prof-specialty`, частично `Tech-support` и `Sales` - более высокие доли `>50k`; `Handlers-cleaners`, `Other-service`, `Machine-op-inspct` - преимущественно `<=50k` (см. `bar_occupation.png`, `bar_occupation_vs_income.png`).
5. Пол: мужчины чаще в группе `>50k`, женщины - `<=50k` (см. `bar_sex.png`, `bar_sex_vs_income.png`).
6. Раса: распределения по доходу различаются, но эффект меньше, чем у образования/семейного положения/профессии (см. `bar_race.png`, `bar_race_vs_income.png`).
7. `workclass`: частный сектор и госслужба показывают лучшие пропорции `>50k`, `Without-pay`/`Never-worked` - почти всегда `<=50k` (см. `bar_workclass.png`, `bar_workclass_vs_income.png`).
8. `native-country`: США доминируют по числу наблюдений; у ряда стран малые выборки, что ограничивает статистическую интерпретацию (см. `bar_native_country.png`).
9. Источник/раздел (`source`): распределения соответствуют ожиданиям по train/test-сплиту (см. `bar_source.png`).

**Ассоциации между признаками:**
1. На матрице Cramér’s V заметны связи: `marital_status <-> relationship`, а также кластеры занятости `occupation <-> workclass` и их связь с уровнем образования (см. `cramers_v_matrix.png`). Это указывает на частичную избыточность категориальных признаков и потенциальную мультиколлинеарность после one-hot кодирования.
2. Для числовых признаков сильных линейных корреляций нет, что снижает риск коллинеарности в линейных моделях (см. `corr_numeric.png`).

**Ключевые предикторы и направленность эффектов:**
1. Наиболее информативные признаки по направленной ассоциации с `>50k`: наличие `capital_gain` > 0, высокий `education_num`, категории профессий `Exec-maangerial`/`Prof-specialty`, статус `Married-civ-spouse`, повышенные `hours-per-week`.
2. Умеренно информативные: `sex` (мужчина), определенные категории `workclass` и `relationship`.
3. Слабые/спорные: `fnlwgt` и "длинные хвосты" `capital-loss` без явной устойчивости эффекта.
4. Влияние возраста нелинейно: рост вероятности `>50k` до среднего возраста с последующей стабилизацией/плато (см. `hist_age.png`, `box_age_vs_income.png`).

**Импликации для моделирования:**
1. Требуется обработка пропусков в `workclass`/`occupation`/`native_country` и укрупнение редких категорий (особенно в `native-country`) для устойчивости оценок.
2. Целевая несбалансированность диктует необходимость стратифицированной валидации и метрик, устойчивых к дисбалансу.
3. Высокая разреженность `capital_gain`/`capital_loss` подсказывает бинаризацию признаков наличия/отсутствия и возможную лог-трансформацию для ненулевых значений.
4. Категориальные кластеры (`occupation`-`workclass`-`education`) рекомендуют регуляризацию и контроль ширины one-hot пространства; деревья/градиентный бустинг могут естественно учесть нелинейности и взаимодействия.
5. `fnlwgt` выглядит слабополезным и может быть исключен на этапе отбора признаков без потери качества.

**Выводы:**
1. Доход `>50k` связан прежде всего с человеческим капиталом и профилем занятости: образование, тип работы/профессии, семейный статус, нагрузка по часам, а также наличие инвестиционных доходов (`capital_gain`).
2. Гендерные различия и отдельные демографические признаки проявляются, но их вклад уступает образованию/профессии/часам.
3. Качество данных достаточное для моделирования после адресной обработки пропусков и редких категорий; ожидается выигрыш от моделей, умеющих работать с нелинейностями и редкими, но информативными сигналами.